In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Creating the Prediction Dataset

- In this notebook, we aggregate the processed COVID-19 data with the demographic data, and create new rows in an augmented dataset including 14-day windows of COVID-19 cases from mid-March as distinct examples.

In [2]:
demographics = pd.read_csv("../processed_data/demographic_data.csv", converters = {"fips": lambda x: str(x)})
demographics["fips"] = demographics.fips.str.rjust(5, fillchar = "0")

elections = pd.read_csv("../processed_data/elections.csv", converters = {"FIPS": lambda x: str(x)}).rename(columns = {"FIPS": "fips"})

restrictions = pd.read_csv("../processed_data/restriction.csv").rename(columns = {"Date": "date", "RegionName": "state", "C4_Restrictions on gatherings": "c4", "C6_Stay at home requirements": "c6", "StringencyIndex": "stringency"})
restrictions["date"] = restrictions.date.str.slice(5)

protests = pd.read_csv("../processed_data/protests.csv", converters = {"fips": lambda x: str(x)}).rename(columns = {"EVENT_DATE": "date"})
protests["date"] = protests.date.str.slice(5)
protests["protest_size"] = protests["99 or less"] + 2 * protests["100 to 499"] + 3 * protests["500 to 999"] + 4 * protests["1000 to 4999"] + 5 * protests["more than 4999"]
protests = protests[["fips", "date", "protest_size"]]
protests = protests.groupby(["fips", "date"]).max().reset_index()

covid = pd.read_csv("../processed_data/new_confirmed.csv", converters = {"FIPS": lambda x: str(x)}).rename(columns = {"FIPS": "fips"})
covid["fips"] = covid.fips.str.rjust(5, fillchar = "0")
covid.columns = covid.columns.str.slice(stop = 5)
date_names = covid.columns[14:]
# covid = pd.concat([covid.fips, (covid.iloc[:, 1:].rolling(7, axis = 1).mean().iloc[:, 6:])], axis = 1)
# covid = pd.melt(covid, id_vars = ['fips'], value_vars = covid.columns[1:], var_name = "date", value_name = "confirmed_cases")

In [3]:
WINDOW_SIZE = 14

date_cols = covid.columns[1:]
nondate_cols = covid.columns[0:1]

augmented_data = []
augmented_index = []
    
for index, row in tqdm(covid.iterrows()):
    for i, col in enumerate(date_cols[WINDOW_SIZE - 1:]):
        series = row[date_cols[i:i + WINDOW_SIZE]].reset_index(drop=True)
        series_dict = {f"{WINDOW_SIZE - 1 - k}_before": v for k, v in series.to_dict().items()}
        series_dict.update(row[nondate_cols].to_dict())
        augmented_data.append(series_dict)
        augmented_index.append([row['fips'], col])
        
covid_mod = pd.concat([pd.DataFrame(data = augmented_data), pd.DataFrame(data = augmented_index, columns = ["fips2", "date"])], axis = 1).drop("fips2", axis = 1)

2974it [12:22,  4.01it/s]


In [4]:
covid_mod

,13_before,12_before,11_before,10_before,9_before,8_before,7_before,6_before,5_before,4_before,3_before,2_before,1_before,0_before,fips,date
0,6.0,4.0,4.0,3.0,4.0,4.0,0.0,2.0,7.0,7.0,2.0,11.0,14.0,18.0,10001,04-05
1,4.0,4.0,3.0,4.0,4.0,0.0,2.0,7.0,7.0,2.0,11.0,14.0,18.0,42.0,10001,04-06
2,4.0,3.0,4.0,4.0,0.0,2.0,7.0,7.0,2.0,11.0,14.0,18.0,42.0,19.0,10001,04-07
3,3.0,4.0,4.0,0.0,2.0,7.0,7.0,2.0,11.0,14.0,18.0,42.0,19.0,54.0,10001,04-08
4,4.0,4.0,0.0,2.0,7.0,7.0,2.0,11.0,14.0,18.0,42.0,19.0,54.0,13.0,10001,04-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99999,12-03
734574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99999,12-04
734575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99999,12-05
734576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99999,12-06


In [5]:
combined = demographics.merge(elections, on = "fips")
combined = pd.concat([combined.assign(date = d) for d in date_names], ignore_index = True)
combined = combined.merge(restrictions, on = ["date", "state"])
combined = combined.merge(protests, how = "left", on = ["date", "fips"]).fillna(value = {"protest_size": -1})
combined = combined.merge(covid_mod, on = ["date", "fips"])
combined = combined.set_index(["date", "state", "fips"])
combined

county state_code    male  female  \
date  state      fips                                                   
04-05 Delaware   10001         Kent County         DE   84271   90551   
                 10003   New Castle County         DE  268870  286263   
                 10005       Sussex County         DE  106429  113111   
      Florida    12001      Alachua County         FL  127298  135850   
                 12003        Baker County         FL   14753   13032   
...                                    ...        ...     ...     ...   
12-07 Washington 53069    Wahkiakum County         WA    2000    2189   
                 53071  Walla Walla County         WA   30830   29406   
                 53073      Whatcom County         WA  107228  109584   
                 53075      Whitman County         WA   24749   23844   
                 53077       Yakima County         WA  124900  124425   

                        median_age  population  female_percentage        lat  \
date  state      fips                                                          
04-05 Delaware   10001        37.6      174822          51.796113  39.086169   
                 10003        38.1      555133          51.566562  39.576833   
                 10005        49.0      219540          51.521818  38.660553   
      Florida    12001        31.3      263148          51.624941  29.674750   
                 12003        37.6       27785          46.903005  30.331098   
...                            ...         ...                ...        ...   
12-07 Washington 53069        54.3        4189          52.255908  46.291769   
                 53071        37.1       60236          48.817983  46.229773   
                 53073        37.0      216812          50.543328  48.825909   
                 53075        24.6       48593          49.068796  46.901173   
                 53077        32.8      249325          49.904743  46.457069   

                              long  life_expectancy  ...  9_before  8_before  \
date  state      fips                                ...                       
04-05 Delaware   10001  -75.568422        77.536467  ...       4.0       4.0   
                 10003  -75.652692        78.985449  ...      19.0      25.0   
                 10005  -75.390038        78.763504  ...      10.0      22.0   
      Florida    12001  -82.357714        78.641544  ...       8.0      10.0   
                 12003  -82.284629        75.251197  ...       2.0       0.0   
...                            ...              ...  ...       ...       ...   
12-07 Washington 53069 -123.424420        78.932648  ...       4.0       1.0   
                 53071 -118.478440        79.796321  ...      28.0      37.0   
                 53073 -121.719892        80.979505  ...      19.0      14.0   
                 53075 -117.523027        81.396547  ...      19.0      14.0   
                 53077 -120.738456        78.437188  ...      79.0     101.0   

                        7_before  6_before  5_before  4_before  3_before  \
date  state      fips                                                      
04-05 Delaware   10001       0.0       2.0       7.0       7.0       2.0   
                 10003      11.0      15.0      41.0      29.0      19.0   
                 10005       7.0      15.0       7.0      13.0       4.0   
      Florida    12001       4.0       8.0       8.0       3.0       5.0   
                 12003       0.0       1.0       1.0       0.0       2.0   
...                          ...       ...       ...       ...       ...   
12-07 Washington 53069       0.0       2.0       0.0       0.0       1.0   
                 53071      34.0      18.0      39.0      25.0      26.0   
                 53073      29.0      83.0      76.0      11.0      39.0   
                 53075       9.0       8.0      27.0      17.0      27.0   
                 53077     105.0      77.0     170.0      61.0      58.0   

               

In [6]:
combined.to_csv("../processed_data/combined.csv")